In [1]:
# test_input = """#.#####
# #.....#
# #>....#
# #.....#
# #...v.#
# #.....#
# #####.#"""

test_input = """#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#"""

In [2]:
import math
import numpy as np
from collections import deque

The entire board cycles with period len(board)

In [3]:
# rows = test_input.split("\n")
rows = open("inputs/24").read().split("\n")

In [4]:
rows

['#.####################################################################################################',
 '#<<<..>>><v<><><v^<><^vv^<<v<^<>>.v^vvvv<v.>v^<<v>>.v>><..<v^>v>.v^v>.><>>^><<.vv<^<>>^v<^.<>>>vvv^v<#',
 '#>><vv.^v>>v>v.v^^^.><^<>>v.^>v^>v><.^>><>>^^.vvv^^^>^>^<vvv.<^.>.><>v.>vv<v.^>>^<><>><>v<v^.>^^^<.<<#',
 '#>>v<v>vv<^^^^^<<v><<^v>^<vv^.v>>.>vv^v<^^..v^>><<>^>v>v<^v.><^<^^v><^^vv<.v..<^<.>^>^<^^^><^<v<^>>v>#',
 '#<v<<<v<.^^^<>v^^^v<v^<<>.v<><v.<<<<><v^>>vv>^>v.^v..<><.<><^.<v^>^<^^<^<v.v.^<v><>v>..>>^^>>^^vv>v>>#',
 '#>^v<><>vvv>>>v^^<^v<v<<<^<.<v^v^^.vvvvv<>vvvv^>v<v^v><<>...>v<^>^.^^^<><^.v<^v>^<v^^<<v^<^<>vv^>^><>#',
 '#<<<<><>^<>><v<^^^>^>v<.^<^<.v<.><><<v<^^><v><>^^.^>>>v^<v^><^^.^<<<.v^>^^<^>^><^^.^v><v^>^<vv^v^<<><#',
 '#>^>v><<<>>.v.>v<^^v<.vvv^vv>>v^v^^><^><>>.>^>^v^v><^v.v<^<v<<v<vv^v<^<<><<v><..^^<.v.v><^<.><>^>>>v<#',
 '#<^^v<.<.vv>>vv<.<.<>><v^^>vv><v<>v^.v>v<.>v^>>>^.v>^<>v<<^vv>^<v^>^><^<v^^<v><>v.>vv^v<<v.vv^>v^...<#',
 '#>^v>>><>^.^<v^<^<^<>>v<>^>>v><^<<v

In [5]:
num_rows, num_cols = len(rows)-2, len(rows[0])-2

In [6]:
num_rows, num_cols

(35, 100)

In [7]:
order = math.lcm(num_rows, num_cols)
order

700

In [8]:
starting_positions = []

for i, r in enumerate(rows[1:-1]):
    for j, c in enumerate(r[1:-1]):
        position = np.array([i, j])

        match c:
            case '>':
                d = 0
            case 'v':
                d = 1
            case '<':
                d = 2
            case '^':
                d = 3
            case '.':
                continue
            case _:
                assert False
        
        starting_positions.append((position, d))

In [9]:
len(starting_positions)

3141

In [10]:
dirs = [np.array([0, 1]), np.array([1, 0]), np.array([0, -1]), np.array([-1, 0])]
dirs_to_axis = [1, 0, 1, 0]
axis_limits = [num_rows, num_cols]

In [11]:
volume = np.zeros((order, num_rows, num_cols), dtype=bool)

for o in range(order):
    for p, d in starting_positions:
        new_position = p + o*dirs[d]
        axis = dirs_to_axis[d]
        new_position[axis] %= axis_limits[axis]

        volume[o, *new_position] = True

In [12]:
lower_bounds = np.array([0, 0])
upper_bounds = np.array([num_rows-1, num_cols-1])

In [13]:
check_dirs = dirs + [np.array([0, 0])]

In [38]:
def walk(start, end, cycle_start=0):
    '''BFS, with wrapping'''
    marked = set([(0, start)])
    to_visit = deque([(0, start)])

    while to_visit:
        t, p = to_visit.popleft()

        if p == end:
            return t

        board = volume[(cycle_start+1) % order]
        next_board = volume[(cycle_start+t+1) % order]

        for delta in check_dirs:
            new_pos = (p[0]+delta[0], p[1]+delta[1])
            n = (t+1, new_pos)

            is_terminal = (new_pos == start) or (new_pos == end)

            bounded = all(x >= y for (x, y) in zip(new_pos, lower_bounds)) and all(x <= y for (x, y) in zip(new_pos, upper_bounds))

            if n not in marked and (is_terminal or (bounded and not next_board[new_pos])):
                marked.add(n)
                to_visit.append(n)

In [39]:
start = (-1, 0)
end = (num_rows, num_cols-1)

In [40]:
best = walk(start, end)
best

242

In [41]:
bt = walk(end, start, best)
ft = walk(start, end, best + bt)

best + bt + ft

720